In [4]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from datasets import load_dataset

In [9]:
import os
os.environ["HF_HUB_READ_TIMEOUT"] = "60"
os.environ["HF_HUB_CONNECT_TIMEOUT"] = "60"

In [10]:
train_dataset = load_dataset('slegroux/tiny-imagenet-200-clean', split='train')                
valid_dataset = load_dataset('slegroux/tiny-imagenet-200-clean', split='validation')
test_dataset = load_dataset('slegroux/tiny-imagenet-200-clean', split='test')

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/151M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/7.54M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/7.57M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/98179 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4909 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4923 [00:00<?, ? examples/s]

In [15]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in dataloader:
        images = batch['image'].to(device)
        label = batch['label'].to(device)

        # Forward
        output = model(images)
        loss = criterion(output, label)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * images.size(0)
        _, preds = output.max(1)
        correct += preds.eq(label).sum().item()
        total += label.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

@torch.no_grad()
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in dataloader:
        images = batch['image'].to(device)
        label = batch['label'].to(device)

        # Forward
        output = model(images)
        loss = criterion(output, label)
        running_loss += loss.item() * images.size(0)
        _, preds = output.max(1)
        correct += preds.eq(label).sum().item()
        total += label.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [17]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.downsample = downsample
        self.strdide = stride

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [18]:
class ResNet34(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.in_channels = 64

        # Initial layers
        self.conv1 = nn.Conv2d(3,64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Stage 1: 3 Blocks, 64 channels
        self.layer1 = self._make_layer(64,3, stride=1)
        # Stage 2: 4 blocks, 128 channels
        self.layer2 = self._make_layer(128, 4, stride=2)
        # Stage 3: 6 blocks, 256 channels
        self.layer3 = self._make_layer(256, 6, stride=2)
        # Stage 4: 3 blocks, 512 channels
        self.layer4 = self._make_layer(512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        downsample = None

        if stride != 1 or self.in_channels != out_channels * BasicBlock.expansion:
            downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                                      nn.BatchNorm2d(out_channels*BasicBlock.expansion)
                                      )
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride, downsample))

        self.in_channels = out_channels * BasicBlock.expansion
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

            
            

In [19]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # AlexNetPCAJitter(IMAGENET_EIGVAL, IMAGENET_EIGVEC),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def preprocess(example, transform):
    example["image"] = [transform(img) for img in example["image"]]
    return example

train_dataset = train_dataset.with_transform(lambda x: preprocess(x, transform))
valid_dataset = valid_dataset.with_transform(lambda x: preprocess(x, test_transform))
test_dataset = test_dataset.with_transform(lambda x: preprocess(x, test_transform))

In [20]:
# del model, optimizer
# torch.cuda.empty_cache()
model_resnet34 = ResNet34(num_classes=200)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet34.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet34.parameters(),lr=0.1, momentum=0.9, weight_decay=0.5e-4)

In [21]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader   = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=4)
test_loader   = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [22]:
best_val_acc = 0.0
num_epochs = 40
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(
        model_resnet34, train_loader, optimizer, criterion, device
    )

    val_loss, val_acc = validate(
        model_resnet34, val_loader, criterion, device
    )

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model_resnet34.state_dict(), "best_model_resnet34.pth")

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
    )


Epoch [1/40] Train Loss: 4.8945, Train Acc: 0.0379 | Val Loss: 4.7157, Val Acc: 0.0448
Epoch [2/40] Train Loss: 4.1351, Train Acc: 0.1192 | Val Loss: 3.7995, Val Acc: 0.1546
Epoch [3/40] Train Loss: 3.5236, Train Acc: 0.2129 | Val Loss: 5.4268, Val Acc: 0.1793
Epoch [4/40] Train Loss: 3.0970, Train Acc: 0.2855 | Val Loss: 3.1482, Val Acc: 0.2844
Epoch [5/40] Train Loss: 2.7437, Train Acc: 0.3527 | Val Loss: 2.6839, Val Acc: 0.3671
Epoch [6/40] Train Loss: 2.4838, Train Acc: 0.4020 | Val Loss: 3.3542, Val Acc: 0.3650
Epoch [7/40] Train Loss: 2.2804, Train Acc: 0.4435 | Val Loss: 2.4990, Val Acc: 0.4125
Epoch [8/40] Train Loss: 2.1011, Train Acc: 0.4802 | Val Loss: 2.5081, Val Acc: 0.4231
Epoch [9/40] Train Loss: 1.9660, Train Acc: 0.5098 | Val Loss: 2.2250, Val Acc: 0.4714
Epoch [10/40] Train Loss: 1.8490, Train Acc: 0.5335 | Val Loss: 2.3363, Val Acc: 0.4636
Epoch [11/40] Train Loss: 1.7388, Train Acc: 0.5581 | Val Loss: 2.2360, Val Acc: 0.4716
Epoch [12/40] Train Loss: 1.6472, Train A

In [23]:
_, test_acc = validate( model_resnet34, test_loader, criterion, device)
print(test_acc)

0.48669510461100957
